In [1]:
import math
import random

In [2]:
def defensive_heuristic1(color, bl, wl):
    if color is -1:
        return 2*len(wl) + random.random()
    else:
        return 2*len(bl) + random.random()


In [3]:
def offensive_heuristic1(color, bl, wl):
    if color is -1:
        return 2*(30-len(bl)) + random.random()
    else:
        return 2*(30-len(wl)) + random.random()


In [4]:
def defensive_heuristic2(color, bl, wl):
    val = 0
    if color is -1:   # white
        for b in bl:
            val +=  7 - b[0]   
        return val + 2*len(wl) + random.random()
    else:             # black
        for w in wl:
            val +=  w[0]   
        return val + 2*len(bl) + random.random()

In [5]:
def offensive_heuristic2(color, bl, wl):
    val = 0
    if color is -1:   # white
        for w in wl:
            val +=  7 - w[0]
        return val + 2*(30-len(bl)) + random.random()
    else:             # black
        for b in bl:
            val +=  b[0]
        return val + 2*(30-len(wl)) + random.random()

In [6]:
def init_pieces():
    wl = set()
    bl = set()
    for i in range(8):
        bl.add((0, i))
        bl.add((1, i))
        wl.add((6, i))
        wl.add((7, i))
    return bl, wl

In [7]:
bl, wl = init_pieces()
node_expanded = 0

In [8]:
# returns boolean, and the piece eaten
def viable(color, new_pos, straight, bl, wl):
    if color == 1:  # black moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 8 and new_pos[1] < 8:
            # check whether moving onto black piece
            if new_pos in bl:
                return False, False
            
            # check whether moving onto white piece
            if straight:
                if new_pos in wl:
                    return False, False
            else: # may eat some piece
                if new_pos in wl:
                    return True, True, new_pos
                
            return True, False
        
        else:
            return False, False
    
    else:           # white moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 8 and new_pos[1] < 8:
            # check whether moving onto white piece
            if new_pos in wl:
                return False, False
            
            # check whether moving onto black piece
            if straight:
                if new_pos in bl:
                    return False, False
            else: # may eat some piece
                if new_pos in bl:
                    return True, True, new_pos
                
            return True, False
        
        else:
            return False, False

In [9]:
def base_case(bl, wl):
    if (not wl) or (not bl):
        #print(wl)
        #print(bl)
        return True
    for p in bl:
        if p[0] == 7:
            return True
    for p in wl:
        if p[0] == 0:
            return True
    
    return False

In [58]:
# black --> 1
# white --> -1

def maxmove(color, bl, wl, depth, init_color, a, b):

    # base case: all pieces of one agent die   or   one pieces of a agent reach the other side
    if base_case(bl, wl) or depth == 0:
        if (init_color == 1):   # heuristic for black
            return defensive_heuristic1(init_color, bl, wl), -1    # the second thing return is a dummy
        else:                   # heursitc for white
            return defensive_heuristic1(init_color, bl, wl), -1    # the second thing return is a dummy 
    
    global node_expanded
    node_expanded += 1
    
    v = -math.inf
    if color == 1:
        cur_max = -math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = minmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
            
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = minmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])                
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
                
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, True, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                val = minmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
                
            
           
        # return the most promising result after loop through all pieces
        return cur_max, best_action
    else:  # color = -1
        cur_max = -math.inf
        best_action = (1, bl, wl)
        # also record best action
        for p in wl:
            
            # left forward
            new_pos = (p[0]-1, p[1]-1)
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = minmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
            
            
            # right forward
            new_pos = (p[0]-1, p[1]+1)        
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = minmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
            
            # forward
            new_pos = (p[0]-1, p[1])
            via = viable(-1, new_pos, True, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                val = minmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
                
           
        # return the most promising result after loop through all pieces
        return cur_max, best_action

In [59]:
# black --> 1
# white --> -1

def minmove(color, bl, wl, depth, init_color, a, b):
    # base case: all pieces of one agent die   or   one pieces of a agent reach the other side
    if base_case(bl, wl) or depth == 0:
        if (init_color == 1):   # heuristic for black
            return defensive_heuristic1(init_color, bl, wl), -1    # the second thing return is a dummy
        else:                   # heursitc for white
            return defensive_heuristic1(init_color, bl, wl), -1    # the second thing return is a dummy 
    
    global node_expanded
    node_expanded += 1
    
    v = math.inf
    
    if color == 1:
        cur_min = math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                       
                    
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
                
                
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, True, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
                
             
           
        # return the most promising result after loop through all pieces
        return cur_min, best_action
    else:  # color = -1
        cur_min = math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in wl:
            
            # left forward
            new_pos = (p[0]-1, p[1]-1)
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = maxmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
            
            
            # right forward
            new_pos = (p[0]-1, p[1]+1)        
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = maxmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
            
            # forward
            new_pos = (p[0]-1, p[1])
            via = viable(-1, new_pos, True, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                val = maxmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
                
           
        # return the most promising result after loop through all pieces
        return cur_min, best_action

In [72]:
bl, wl = init_pieces()
color = 1
counter = 0
node_expanded = 0
while not base_case(bl, wl):
    _, bl, wl = maxmove(color, bl, wl, 3, color, -math.inf, math.inf)[1]
    print(counter)
    board = [[0 for x in range(8)] for y in range(8)]
    update_board(bl, wl)
    print_board()
    color = -color
    counter += 1
    
if (not wl):
    print("Black wins")
elif (not bl):
    print("White wins")
else:    
    
    for p in wl:
        if p[0] == 0:
            print("White wins")
    for p in bl:
        if p[0] == 7:
            print("Black wins")
            
print(node_expanded)
print(node_expanded / counter)

board = [[0 for x in range(8)] for y in range(8)]
update_board(bl, wl)
print_board()
    
    


0
---------------------------------
| w | w | w | w | w | w | w | w |
---------------------------------
| w | w | w | w | w | w | w | 0 |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | w |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------------------
| b | b | b | b | b | b | b | b |
---------------------------------
| b | b | b | b | b | b | b | b |
---------------------------------
1
---------------------------------
| w | w | w | w | w | w | w | w |
---------------------------------
| w | w | w | w | w | w | w | 0 |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | w |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | b | 0 | 0 | 0 | 0 | 0 | 0 |
----------

| b | b | b | b | b | b | b | 0 |
---------------------------------
16
---------------------------------
| w | w | w | 0 | w | w | w | w |
---------------------------------
| w | 0 | w | w | w | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | w | 0 | 0 | 0 | w |
---------------------------------
| 0 | b | w | w | 0 | 0 | 0 | w |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | b | 0 | b | b | b |
---------------------------------
| 0 | b | b | 0 | b | 0 | b | 0 |
---------------------------------
| b | b | b | b | b | b | b | 0 |
---------------------------------
17
---------------------------------
| w | w | w | 0 | w | w | w | w |
---------------------------------
| w | 0 | w | w | w | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | w | 0 | 0 | 0 | w |
---------------------------------
| 0 | b | w | w | 0 | 0 | 0 | w |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
--------

| b | 0 | b | b | 0 | 0 | 0 | 0 |
---------------------------------
31
---------------------------------
| w | 0 | 0 | 0 | w | w | 0 | w |
---------------------------------
| w | w | w | w | w | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | w | 0 | 0 | 0 | w |
---------------------------------
| 0 | b | w | w | 0 | w | w | w |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | b | b | b | b | b | b | b |
---------------------------------
| 0 | b | b | b | b | b | b | 0 |
---------------------------------
| b | 0 | b | 0 | 0 | 0 | 0 | 0 |
---------------------------------
32
---------------------------------
| w | 0 | 0 | 0 | w | w | 0 | w |
---------------------------------
| w | w | w | w | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | 0 | 0 | w | w | 0 | 0 | w |
---------------------------------
| 0 | b | w | w | 0 | w | w | w |
---------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
--------

| 0 | b | b | 0 | w | b | b | 0 |
---------------------------------
| b | 0 | b | 0 | 0 | 0 | 0 | 0 |
---------------------------------
48
---------------------------------
| 0 | 0 | 0 | 0 | 0 | w | 0 | 0 |
---------------------------------
| w | w | w | w | b | w | w | 0 |
---------------------------------
| 0 | w | 0 | w | w | 0 | 0 | w |
---------------------------------
| 0 | b | 0 | 0 | 0 | w | 0 | w |
---------------------------------
| 0 | w | b | b | 0 | 0 | 0 | 0 |
---------------------------------
| 0 | b | 0 | b | b | 0 | b | b |
---------------------------------
| 0 | b | b | 0 | w | b | b | 0 |
---------------------------------
| b | 0 | b | 0 | 0 | 0 | 0 | 0 |
---------------------------------
49
---------------------------------
| 0 | 0 | 0 | 0 | 0 | w | 0 | 0 |
---------------------------------
| w | w | w | w | b | w | w | 0 |
---------------------------------
| 0 | w | 0 | w | w | 0 | 0 | w |
---------------------------------
| 0 | b | 0 | 0 | 0 | w | 0 | w |
--------

In [ ]:
blackwins = 0
whitewins = 0
for i in range(25):
    bl, wl = init_pieces()
    color = 1
    counter = 0
    node_expanded = 0
    while not base_case(bl, wl):
        _, bl, wl = maxmove(color, bl, wl, 3, color, -math.inf, math.inf)[1]
        color = -color
        counter += 1

    if (not wl):
        print("Black wins")
        blackwins += 1
    elif (not bl):
        print("White wins")
        whitewins += 1
    else:    
        for p in wl:
            if p[0] == 0:
                print("White wins")
                whitewins += 1 
        for p in bl:
            if p[0] == 7:
                print("Black wins")
                blackwins += 1
                
    print("blackwins", blackwins, "whitewins", whitewins)

Black wins
blackwins 1 whitewins 0
Black wins
blackwins 2 whitewins 0
Black wins
blackwins 3 whitewins 0
Black wins
blackwins 4 whitewins 0
Black wins
blackwins 5 whitewins 0
Black wins
blackwins 6 whitewins 0
Black wins
blackwins 7 whitewins 0
Black wins
blackwins 8 whitewins 0
Black wins
blackwins 9 whitewins 0
Black wins
blackwins 10 whitewins 0
Black wins
blackwins 11 whitewins 0
Black wins
blackwins 12 whitewins 0
Black wins
blackwins 13 whitewins 0
Black wins
blackwins 14 whitewins 0
Black wins
blackwins 15 whitewins 0
Black wins
blackwins 16 whitewins 0
Black wins
blackwins 17 whitewins 0
Black wins
blackwins 18 whitewins 0
Black wins
blackwins 19 whitewins 0
Black wins
blackwins 20 whitewins 0
Black wins
blackwins 21 whitewins 0
Black wins
blackwins 22 whitewins 0


In [ ]:
# black --> 1
# white --> -1

def greedy(color, bl, wl):
    global node_expanded
    node_expanded += 1
    
    v = -math.inf       # current biggest value
    new_bl = bl.copy()
    new_wl = wl.copy()
    
    if color == 1:        #black        
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                temp_new_bl = bl.copy()
                temp_new_wl = wl.copy()
                temp_new_bl.remove(p)
                temp_new_bl.add(new_pos)
                if via[1]:
                    temp_new_wl.remove(via[2])
                heuristic = defensive_heuristic1(color, bl, wl)
                if heuristic > v:
                    v = heuristic
                    new_bl = temp_new_bl.copy()
                    new_wl = temp_new_wl.copy()
            
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                temp_new_bl = bl.copy()
                temp_new_wl = wl.copy()
                temp_new_bl.remove(p)
                temp_new_bl.add(new_pos)
                if via[1]:
                    temp_new_wl.remove(via[2])
                heuristic = defensive_heuristic1(color, bl, wl)
                if heuristic > v:
                    v = heuristic
                    new_bl = temp_new_bl.copy()
                    new_wl = temp_new_wl.copy()
                
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                temp_new_bl = bl.copy()
                temp_new_wl = wl.copy()
                temp_new_bl.remove(p)
                temp_new_bl.add(new_pos)
                if via[1]:
                    temp_new_wl.remove(via[2])
                heuristic = defensive_heuristic1(color, bl, wl)
                if heuristic > v:
                    v = heuristic
                    new_bl = temp_new_bl.copy()
                    new_wl = temp_new_wl.copy()
            
           
        # return the most promising result after loop through all pieces
        return v, (color, new_bl, new_wl)
    
    else:        #color = -1. white        
        for p in wl:
            
            # left forward
            new_pos = (p[0]-1, p[1]-1)
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                temp_new_bl = bl.copy()
                temp_new_wl = wl.copy()
                temp_new_wl.remove(p)
                temp_new_wl.add(new_pos)
                if via[1]:
                    temp_new_bl.remove(via[2])
                heuristic = defensive_heuristic1(color, bl, wl)
                if heuristic > v:
                    v = heuristic
                    new_bl = temp_new_bl.copy()
                    new_wl = temp_new_wl.copy()            
            
            
            # right forward
            new_pos = (p[0]-1, p[1]+1)        
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                temp_new_bl = bl.copy()
                temp_new_wl = wl.copy()
                temp_new_wl.remove(p)
                temp_new_wl.add(new_pos)
                if via[1]:
                    temp_new_bl.remove(via[2])
                heuristic = defensive_heuristic1(color, bl, wl)
                if heuristic > v:
                    v = heuristic
                    new_bl = temp_new_bl.copy()
                    new_wl = temp_new_wl.copy()
                
            # forward
            new_pos = (p[0]-1, p[1])     
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                temp_new_bl = bl.copy()
                temp_new_wl = wl.copy()
                temp_new_wl.remove(p)
                temp_new_wl.add(new_pos)
                if via[1]:
                    temp_new_bl.remove(via[2])
                heuristic = defensive_heuristic1(color, bl, wl)
                if heuristic > v:
                    v = heuristic
                    new_bl = temp_new_bl.copy()
                    new_wl = temp_new_wl.copy()
            
           
        # return the most promising result after loop through all pieces
        return v, (color, new_bl, new_wl)
    

In [65]:
blackwins = 0
whitewins = 0
for i in range(20):
    bl, wl = init_pieces()
    color = 1
    counter = 0
    node_expanded = 0
    while not base_case(bl, wl):
        if color == -1:     #white
            _, bl, wl = maxmove(color, bl, wl, 3, color, -math.inf, math.inf)[1]
        else:              #black
            _, bl, wl = greedy(color, bl, wl)[1]
        color = -color
        counter += 1

    if (not wl):
        print("Black wins")
        blackwins += 1
    elif (not bl):
        print("White wins")
        whitewins += 1
    else:    
        for p in wl:
            if p[0] == 0:
                print("White wins")
                whitewins += 1 
        for p in bl:
            if p[0] == 7:
                print("Black wins")
                blackwins += 1
                
    print("blackwins", blackwins, "whitewins", whitewins)
    print(wl)
    print(bl)

Black wins
blackwins 1 whitewins 0
{(1, 2), (4, 7), (1, 3), (6, 7), (6, 1), (3, 1), (3, 7), (5, 1)}
{(3, 2), (0, 0), (4, 4), (7, 5)}
Black wins
blackwins 2 whitewins 0
{(5, 4), (5, 1), (7, 0), (4, 4), (5, 7), (7, 7), (1, 5), (6, 2), (4, 3), (4, 2), (3, 4), (2, 4)}
{(2, 7), (4, 7), (0, 0), (7, 5), (2, 2), (3, 7), (1, 4)}


KeyboardInterrupt: 

In [63]:
def update_board(wl, bl):
    for b in bl:
        board[b[0]][b[1]] = "b"
    for w in wl:
        board[w[0]][w[1]] = "w"

In [70]:
def print_board():
    print("---------------------------------")
    for i in range(8):
        print("|", board[i][0], "|", board[i][1], "|", board[i][2], "|", board[i][3], "|", board[i][4], "|", board[i][5], "|", board[i][6], "|", board[i][7], "|")
        print("---------------------------------")